# Module 2 - Create Knowledge Base and Ingest Documents

----

This notebook provides sample code with step by step instructions for building an Amazon Bedrock Knowledge Base.

----

### Contents

1. *Setup*
1. *Create an S3 Data Source*
1. *Setup AOSS Vector Index and Configure BKB Access Permissions*
2. *Configure Amazon Bedrock Knowledge Base and Synchronize it with Data Source*
3. *Conclusions and Next Steps*

----

### Background

Foundation models (FMs) are powerful generative AI models trained on vast amounts of general-purpose data. However, many real-world applications require these models to generate responses grounded in domain-specific or proprietary information. Retrieval Augmented Generation (RAG) is a technique that enhances generative AI responses by retrieving relevant information from external data sources at query time.

Bedrock Knowledge Bases (BKBs) provide a fully managed capability to implement RAG-based solutions. By integrating your own data — such as documents, manuals, or knowledge articles — into a knowledge base, you can improve the accuracy, relevance, and usefulness of model-generated responses. When a user submits a query, Amazon Bedrock Knowledge Bases search across the ingested data, retrieve the most relevant content, and pass this information to the foundation model to generate a more informed response.

This notebook demonstrates how to create an empty Amazon OpenSearch Serverless (AOSS) index, build an Amazon Bedrock Knowledge Base, and ingest documents into it for retrieval-augmented generation.

### Pre-requisites
This notebook requires permissions to:
- create and delete **Amazon IAM** roles
- create, update and delete **Amazon S3** buckets
- access **Amazon Bedrock**
- access to **Amazon OpenSearch Serverless**

If running on **SageMaker Studio**, you should add the following managed policies to your role:
- `IAMFullAccess`
- `AWSLambda_FullAccess`
- `AmazonS3FullAccess`
- `AmazonBedrockFullAccess`
- Custom policy for Amazon OpenSearch Serverless such as:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:*",
            "Resource": "*"
        }
    ]
}
```
<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to enable `Anthropic Claude 3 Sonnet` and `Anthropic Claude 3 Haiku` model access in Amazon Bedrock Console, as the notebook will use Anthropic Claude 3 Sonnet and Claude 3 Haiku models for testing the knowledge base once its created.
</div>


## 1. Setup

### 1.1 Install and import the required libraries


In [1]:
%pip install --force-reinstall -q -r ./requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.29.1 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.20.0 requires botocore<1.36.24,>=1.36.20, but you have botocore 1.37.30 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-m

In [2]:
# Restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
# Standard library imports
import os
import sys
import json
import time
import pprint
import random

# Third-party imports
import boto3
from botocore.exceptions import ClientError

# Local imports
import utility

# Print SDK versions
print(f"Python version: {sys.version.split()[0]}")
print(f"Boto3 SDK version: {boto3.__version__}")

Python version: 3.11.11
Boto3 SDK version: 1.37.30


### 1.2 Initial setup for clients and global variables

In [4]:
# Create boto3 session and set AWS region
boto_session = boto3.Session()
aws_region = boto_session.region_name

# Create boto3 clients for AOSS, Bedrock, and S3 services
aoss_client = boto3.client('opensearchserverless')
bedrock_agent_client = boto3.client('bedrock-agent')
s3_client = boto3.client('s3')

# Define names for AOSS, Bedrock, and S3 resources
resource_suffix = random.randrange(100, 999)
s3_bucket_name = f"bedrock-kb-{aws_region}-{resource_suffix}"
aoss_collection_name = f"bedrock-kb-collection-{resource_suffix}" # was: vector_store_name
aoss_index_name = f"bedrock-kb-index-{resource_suffix}"           # was: index_name
bedrock_kb_name = f"bedrock-kb-{resource_suffix}"

# Some temporary local paths
local_data_dir = 'data'

# Print configurations
print("AWS Region:", aws_region)
print("S3 Bucket:", s3_bucket_name)
print("AOSS Collection Name:", aoss_collection_name)
print("Bedrock Knowledge Base Name:", aoss_collection_name)

AWS Region: us-east-1
S3 Bucket: bedrock-kb-us-east-1-744
AOSS Collection Name: bedrock-kb-collection-744
Bedrock Knowledge Base Name: bedrock-kb-collection-744


## 2. Create an S3 Data Source

Amazon Bedrock Knowledge Bases can connect to a variety of data sources for downstream RAG applications. Supported data sources include Amazon S3, Confluence, Microsoft SharePoint, Salesforce, Web Crawler, and custom data sources.

In this workshop, we will use Amazon S3 to store unstructured data — specifically, PDF files containing Amazon Shareholder Letters from different years. This S3 bucket will serve as the source of documents for our Knowledge Base. During the ingestion process, Bedrock will parse these documents, convert them into vector embeddings using an embedding model, and store them in a vector database for efficient retrieval during queries.

### 2.1 Create an S3 bucket, if needed

In [5]:
# Check if bucket exists, and if not create S3 bucket for KB data source

try:
    s3_client.head_bucket(Bucket=s3_bucket_name)
    print(f"Bucket '{s3_bucket_name}' already exists..")
except ClientError as e:
    print(f"Creating bucket: '{s3_bucket_name}'..")
    if aws_region == 'us-east-1':
        s3_client.create_bucket(Bucket=s3_bucket_name)
    else:
        s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
        )

Creating bucket: 'bedrock-kb-us-east-1-744'..


### 2.2 Download data and upload to S3

In [6]:
from urllib.request import urlretrieve

# URLs of shareholder letters to download
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

# Corresponding local file names
filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

# Create local staging directory if it doesn't exist
os.makedirs(local_data_dir, exist_ok=True)

# Download each file and print confirmation
for url, filename in zip(urls, filenames):
    file_path = os.path.join(local_data_dir, filename)
    urlretrieve(url, file_path)
    print(f"Downloaded: '{filename}' to '{local_data_dir}'..")

Downloaded: 'AMZN-2022-Shareholder-Letter.pdf' to 'data'..
Downloaded: 'AMZN-2021-Shareholder-Letter.pdf' to 'data'..
Downloaded: 'AMZN-2020-Shareholder-Letter.pdf' to 'data'..
Downloaded: 'AMZN-2019-Shareholder-Letter.pdf' to 'data'..


In [7]:
for root, _, files in os.walk(local_data_dir):
    for file in files:
        full_path = os.path.join(root, file)
        s3_client.upload_file(full_path, s3_bucket_name, file)
        print(f"Uploaded: '{file}' to 's3://{s3_bucket_name}'..")

Uploaded: 'AMZN-2022-Shareholder-Letter.pdf' to 's3://bedrock-kb-us-east-1-744'..
Uploaded: 'AMZN-2021-Shareholder-Letter.pdf' to 's3://bedrock-kb-us-east-1-744'..
Uploaded: 'AMZN-2020-Shareholder-Letter.pdf' to 's3://bedrock-kb-us-east-1-744'..
Uploaded: 'AMZN-2019-Shareholder-Letter.pdf' to 's3://bedrock-kb-us-east-1-744'..


Store S3 bucket name for later reuse in other notebooks:

In [8]:
%store s3_bucket_name

Stored 's3_bucket_name' (str)


## 3 Setup AOSS Vector Index and Configure BKB Access Permissions

In this section, we’ll create a vector index using Amazon OpenSearch Serverless (AOSS) and configure the necessary access permissions for the Bedrock Knowledge Base (BKB) that we’ll set up later. AOSS provides a fully managed, serverless solution for running vector search workloads at billion-vector scale. It automatically handles resource scaling and eliminates the need for cluster management, while delivering low-latency, millisecond response times with pay-per-use pricing.

While this example uses AOSS, it’s worth noting that Bedrock Knowledge Bases also supports other popular vector stores, including Amazon Aurora PostgreSQL with pgvector, Pinecone, Redis Enterprise Cloud, and MongoDB, among others

### 3.1 Create IAM Role with Necessary Permissions for Bedrock Knowledge Base

Let's first create an IAM role with all the necessary policies and permissions to allow BKB to execute operations, such as invoking Bedrock FMs and reading data from an S3 bucket. We will use a helper function for this.

In [9]:
bedrock_kb_execution_role = utility.create_bedrock_execution_role(bucket_name=s3_bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

print("Created KB execution role with ARN:", bedrock_kb_execution_role_arn)

Created KB execution role with ARN: arn:aws:iam::242711262407:role/AmazonBedrockExecutionRoleForKnowledgeBase_849


### 3.2 Create AOSS Policies and Vector Collection

Next we need to create and attach three key policies for securing and managing access to the AOSS collection: an encryption policy, a network access policy, and a data access policy. These policies ensure proper encryption, network security, and the necessary permissions for creating, reading, updating, and deleting collection items and indexes. This step is essential for configuring the OpenSearch collection to interact with BKB securely and efficiently (you can read more about AOSS collections [here](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless.html)). We will use another helper function for this.

⚠️ **Note:** _in order to keep setup overhead at mininum, in this example we **allow public internet access** to the OpenSearch Serverless collection resource. However, for production environments we strongly suggest to leverage private connection between your VPC and Amazon OpenSearch Serverless resources via an VPC endpoint, as described [here](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-vpc.html)._

In [10]:
# Create AOSS policies for the new vector collection
encryption_policy, network_policy, access_policy = utility.create_policies_in_oss(
    vector_store_name=aoss_collection_name,
    aoss_client=aoss_client,
    bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)

print("Created encryption policy with name:", encryption_policy['securityPolicyDetail']['name'])
print("Created network policy with name:", network_policy['securityPolicyDetail']['name'])
print("Created access policy with name:", access_policy['accessPolicyDetail']['name'])

Created encryption policy with name: bedrock-sample-rag-sp-849
Created network policy with name: bedrock-sample-rag-np-849
Created access policy with name: bedrock-sample-rag-ap-849


In [11]:
# Request to create AOSS collection
aoss_collection = aoss_client.create_collection(name=aoss_collection_name, type='VECTORSEARCH')

# Wait until collection becomes active
print("Waiting until AOSS collection becomes active: ", end='')
while True:
    response = aoss_client.list_collections(collectionFilters={'name': aoss_collection_name})
    status = response['collectionSummaries'][0]['status']
    if status in ('ACTIVE', 'FAILED'):
        print(" done.")
        break
    print('█', end='', flush=True)
    time.sleep(5)

print("An AOSS collection created:", json.dumps(response['collectionSummaries'], indent=2))

Waiting until AOSS collection becomes active: █████ done.
An AOSS collection created: [
  {
    "id": "bf0v0114obxb77yn1mye",
    "name": "bedrock-kb-collection-744",
    "status": "ACTIVE",
    "arn": "arn:aws:aoss:us-east-1:242711262407:collection/bf0v0114obxb77yn1mye"
  }
]


Store AOSS-related resouces for access in other notebooks later:

In [12]:
%store encryption_policy network_policy access_policy aoss_collection

Stored 'encryption_policy' (dict)
Stored 'network_policy' (dict)
Stored 'access_policy' (dict)
Stored 'aoss_collection' (dict)


### 3.2 Grant BKB Access to AOSS Data

In this step, we create a data access policy that grants BKB the necessary permissions to read from our AOSS collections. We then attach this policy to the Bedrock execution role we created earlier, allowing BKB to securely access AOSS data when generating responses. We will be using helper function once again.

In [13]:
aoss_policy_arn = utility.create_oss_policy_attach_bedrock_execution_role(
    collection_id=aoss_collection['createCollectionDetail']['id'],
    bedrock_kb_execution_role=bedrock_kb_execution_role)

print("Waiting 60 sec for data access rules to be enforced: ", end='')
for _ in range(12):  # 12 * 5 sec = 60 sec
    print('█', end='', flush=True)
    time.sleep(5)
print(" done.")

print("Created and attached policy with ARN:", aoss_policy_arn)

Waiting 60 sec for data access rules to be enforced: ████████████ done.
Created and attached policy with ARN: arn:aws:iam::242711262407:policy/AmazonBedrockOSSPolicyForKnowledgeBase_849


### 3.3 Create an AOSS Vector Index

Now that we have all necessary access permissions in place, we can create a vector index in the AOSS collection we created previously.

In [14]:
from requests_aws4auth import AWS4Auth
from opensearchpy import OpenSearch, RequestsHttpConnection

# Use default credential configuration for authentication
credentials = boto_session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    aws_region,
    'aoss',
    session_token=credentials.token)

# Construct AOSS endpoint host
host = f"{aoss_collection['createCollectionDetail']['id']}.{aws_region}.aoss.amazonaws.com"

# Build the OpenSearch client
os_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [15]:
# Define the configuration for the AOSS vector index
index_definition = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"
         }
      }
   }
}

# Create AOSS index
response = os_client.indices.create(index=aoss_index_name, body=index_definition)

# Waiting for index creation to propagate
print("Waiting 30 sec for index update to propagate: ", end='')
for _ in range(6):  # 6 * 5 sec = 30 sec
    print('█', end='', flush=True)
    time.sleep(5)
print(" done.")

print("A new AOSS index created:", json.dumps(response, indent=2))

Waiting 30 sec for index update to propagate: ██████ done.
A new AOSS index created: {
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "bedrock-kb-index-744"
}


## 4. Configure Amazon Bedrock Knowledge Base and Synchronize it with Data Source

In this section, we’ll create an Amazon Bedrock Knowledge Base (BKB) and connect it to the data that will be stored in our newly created AOSS vector index.

### 4.1 Create a Bedrock Knowledge Base

Setting up a Knowledge Base involves providing two key configurations:
1. **Storage Configuration** tells Bedrock where to store the generated vector embeddings by specifying the target vector store and providing the necessary connection detail (we use AOSS vector index in our case),
2. **Knowledge Base Configuration** defines how Bedrock should generate vector embeddings from your data by specifying the embedding model to use, along with any additional settings required for handling multimodal content.

In [16]:
# Vector Storage Configuration
storage_config = {
    "type": "OPENSEARCH_SERVERLESS",
    "opensearchServerlessConfiguration": {
        "collectionArn": aoss_collection["createCollectionDetail"]['arn'],
        "vectorIndexName": aoss_index_name,
        "fieldMapping": {
            "vectorField": "vector",
            "textField": "text",
            "metadataField": "text-metadata"
        }
    }
}

# Knowledge Base Configuration
knowledge_base_config = {
    "type": "VECTOR",
    "vectorKnowledgeBaseConfiguration": {
        "embeddingModelArn": f"arn:aws:bedrock:{aws_region}::foundation-model/amazon.titan-embed-text-v1"
    }
}

response = bedrock_agent_client.create_knowledge_base(
    name=bedrock_kb_name,
    description="Amazon shareholder letter knowledge base.",
    roleArn=bedrock_kb_execution_role_arn,
    knowledgeBaseConfiguration=knowledge_base_config,
    storageConfiguration=storage_config)

bedrock_kb_id = response['knowledgeBase']['knowledgeBaseId']

print("Waiting until BKB becomes active: ", end='')
while True:
    response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=bedrock_kb_id)
    if response['knowledgeBase']['status'] == 'ACTIVE':
        print(" done.")
        break
    print('█', end='', flush=True)
    time.sleep(5)

print("A new Bedrock Knowledge Base created with ID:", bedrock_kb_id)

Waiting until BKB becomes active: █ done.
A new Bedrock Knowledge Base created with ID: UMRNTGVWEI


Let's call a Bedrock API to get the information about our newly created Knowledge Base:

In [17]:
response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=bedrock_kb_id)

print(json.dumps(response['knowledgeBase'], indent=2, default=str))

{
  "createdAt": "2025-04-09 14:18:32.588885+00:00",
  "description": "Amazon shareholder letter knowledge base.",
  "knowledgeBaseArn": "arn:aws:bedrock:us-east-1:242711262407:knowledge-base/UMRNTGVWEI",
  "knowledgeBaseConfiguration": {
    "type": "VECTOR",
    "vectorKnowledgeBaseConfiguration": {
      "embeddingModelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1"
    }
  },
  "knowledgeBaseId": "UMRNTGVWEI",
  "name": "bedrock-kb-744",
  "roleArn": "arn:aws:iam::242711262407:role/AmazonBedrockExecutionRoleForKnowledgeBase_849",
  "status": "ACTIVE",
  "storageConfiguration": {
    "opensearchServerlessConfiguration": {
      "collectionArn": "arn:aws:aoss:us-east-1:242711262407:collection/bf0v0114obxb77yn1mye",
      "fieldMapping": {
        "metadataField": "text-metadata",
        "textField": "text",
        "vectorField": "vector"
      },
      "vectorIndexName": "bedrock-kb-index-744"
    },
    "type": "OPENSEARCH_SERVERLESS"
  },
  "updated

### 4.2 Connect KBK to a Data Source

With our Knowledge Base in place, the next step is to connect it to a data source. This involves two key actions:

1. Create a data source for the Knowledge Base that will point to the location of our raw data (in this case, S3),
2. Define how that data should be processed and ingested into the vector store — for example, by specifying a chunking configuration that controls how large each text fragment should be when generating vector embeddings for retrieval.

In [18]:
# Data Source Configuration
data_source_config = {
        "type": "S3",
        "s3Configuration":{
            "bucketArn": f"arn:aws:s3:::{s3_bucket_name}",
            # "inclusionPrefixes":["*.*"]   # you can use this if you want to create a KB using data within s3 prefixes.
        }
    }

# Vector Ingestion Configuration
vector_ingestion_config = {
        "chunkingConfiguration": {
            "chunkingStrategy": "FIXED_SIZE",
            "fixedSizeChunkingConfiguration": {
                "maxTokens": 512,
                "overlapPercentage": 20
            }
        }
    }

response = bedrock_agent_client.create_data_source(
    name=bedrock_kb_name,
    description="Amazon shareholder letter knowledge base.",
    knowledgeBaseId=bedrock_kb_id,
    dataSourceConfiguration=data_source_config,
    vectorIngestionConfiguration=vector_ingestion_config
)

bedrock_ds_id = response['dataSource']['dataSourceId']

print("A new BKB data source created with ID:", bedrock_ds_id)

A new BKB data source created with ID: RNHQ0DDCF8


Let's also use Bedrock API to get the information about our newly created BKB data source:

In [19]:
response = bedrock_agent_client.get_data_source(knowledgeBaseId=bedrock_kb_id, dataSourceId=bedrock_ds_id)

print(json.dumps(response['dataSource'], indent=2, default=str))

{
  "createdAt": "2025-04-09 14:18:38.385297+00:00",
  "dataDeletionPolicy": "DELETE",
  "dataSourceConfiguration": {
    "s3Configuration": {
      "bucketArn": "arn:aws:s3:::bedrock-kb-us-east-1-744"
    },
    "type": "S3"
  },
  "dataSourceId": "RNHQ0DDCF8",
  "description": "Amazon shareholder letter knowledge base.",
  "knowledgeBaseId": "UMRNTGVWEI",
  "name": "bedrock-kb-744",
  "status": "AVAILABLE",
  "updatedAt": "2025-04-09 14:18:38.385297+00:00",
  "vectorIngestionConfiguration": {
    "chunkingConfiguration": {
      "chunkingStrategy": "FIXED_SIZE",
      "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
      }
    }
  }
}


### 4.3 Synchronize BKB with Data Source

Once the Knowledge Base and its data source are configured, we can start a data ingestion job. During this process, BKB will retrieve the documents from the connected data source (on S3, in this case), extract and preprocess the content, split it into smaller chunks based on the configured chunking strategy, generate vector embeddings for each chunk, and store those embeddings in the vector store (AOSS vector store, in this case).

In [20]:
# Start an ingestion job
response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId=bedrock_kb_id, dataSourceId=bedrock_ds_id)

bedrock_job_id = response['ingestionJob']['ingestionJobId']

print("A new BKB ingestion job started with ID:", bedrock_job_id)

A new BKB ingestion job started with ID: SGXMGPCD3X


In [21]:
# Wait until ingestion job completes
print("Waiting until BKB ingestion job completes: ", end='')
while True:
    response = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId = bedrock_kb_id,
        dataSourceId = bedrock_ds_id,
        ingestionJobId = bedrock_job_id)
    if response['ingestionJob']['status'] == 'COMPLETE':
        print(" done.")
        break
    print('█', end='', flush=True)
    time.sleep(5)

print("The BKB ingestion job finished:", json.dumps(response['ingestionJob'], indent=2, default=str))

Waiting until BKB ingestion job completes: ███ done.
The BKB ingestion job finished: {
  "dataSourceId": "RNHQ0DDCF8",
  "ingestionJobId": "SGXMGPCD3X",
  "knowledgeBaseId": "UMRNTGVWEI",
  "startedAt": "2025-04-09 14:18:39.448894+00:00",
  "statistics": {
    "numberOfDocumentsDeleted": 0,
    "numberOfDocumentsFailed": 0,
    "numberOfDocumentsScanned": 4,
    "numberOfMetadataDocumentsModified": 0,
    "numberOfMetadataDocumentsScanned": 0,
    "numberOfModifiedDocumentsIndexed": 0,
    "numberOfNewDocumentsIndexed": 4
  },
  "status": "COMPLETE",
  "updatedAt": "2025-04-09 14:18:49.931337+00:00"
}


Store BKB resouces for access in other notebooks later:

In [22]:
%store bedrock_kb_id

Stored 'bedrock_kb_id' (str)


## 5. Conclusions and Next Steps

In this notebook, we walked through the process of creating an Amazon Bedrock Knowledge Base (BKB) and ingesting documents to enable Retrieval Augmented Generation (RAG) capabilities. We started by setting up the environment, installing the required libraries, and initializing the necessary AWS service clients. Then, we created an Amazon S3 bucket to store unstructured data (PDF documents) and uploaded sample files. We proceeded by provisioning an Amazon OpenSearch Serverless (AOSS) collection and index, configuring the appropriate IAM roles and permissions, and granting access to the BKB. Finally, we created the BKB, connected it to the S3 data source, and synchronized the documents to generate vector embeddings, which were stored in AOSS.

### Next Steps

To explore how to interact with the newly created Knowledge Base via Bedrock APIs for text generation and RAG applications, please proceed to the next notebook: [2_managed-rag-kb-retrieve-generate-api.ipynb](./2\_managed-rag-kb-retrieve-generate-api.ipynb)